In [ ]:
from api_helpers.helpers.logging_config import I, W

: 

In [67]:
import pandas as pd
import numpy as np

from dotenv import load_dotenv

load_dotenv(
    dotenv_path="/Users/tomwattley/App/racing-api-project/racing-api-project/libraries/api-helpers/src/api_helpers/.env"
)

True

In [77]:
from api_helpers.clients.betfair_client import BetFairOrder, OrderResult


class TestS3Client:
    def __init__(self):
        self.stored_data = []

    def store_data(self, data: pd.DataFrame, object_path: str):
        self.stored_data.append({"object_path": object_path, "data": data})


class TestBetfairClient:
    def __init__(self):
        self.cash_out_market_ids = []
        self.placed_orders = []

    def cash_out_bets(self, market_ids: list[str]):
        self.cash_out_market_ids.append(list(market_ids))
        return self.cash_out_market_ids

    def place_order(self, betfair_order: BetFairOrder):
        self.placed_orders.append({"betfair_order": betfair_order})
        return OrderResult(success=True, message="Test Bet Placed")

In [78]:
SELECTION_COLS = [
    "id",
    "timestamp",
    "race_id",
    "race_time",
    "race_date",
    "horse_id",
    "horse_name",
    "selection_type",
    "market_type",
    "market_id",
    "selection_id",
    "requested_odds",
    "valid",
    "invalidated_at",
    "invalidated_reason",
    "size_matched",
    "average_price_matched",
    "cashed_out",
    "fully_matched",
    "customer_strategy_ref",
    "processed_at",
]

# CURRENT IMPLEMENTATION DOESNT ACCOUNT FOR MATCHED INVALID BETS

In [79]:
from trader.fetch_requests import *
from trader.prepare_requests import *
from api_helpers.helpers.data_utils import print_dataframe_for_testing
from api_helpers.helpers.file_utils import S3FilePaths

paths = S3FilePaths()

# betfair_client = get_betfair_client()
# s3_client = get_s3_client()
# betting_data = fetch_betting_data(s3_client, betfair_client)
# data= prepare_request_data(betting_data)

In [80]:
# sf = s3_client.fetch_data('today/2025_05_28/trader_data/selections.parquet')

In [81]:
def create_test_data(requests_overrides=None):
    requests_dict = {
        "id": ["1", "2", "3"],
        "timestamp": [
            pd.Timestamp("2025-01-01 12:00:00"),
            pd.Timestamp("2025-01-01 17:00:00"),
            pd.Timestamp("2025-01-01 18:00:00"),
        ],
        "race_id": [1, 2, 3],
        "race_time": [
            pd.Timestamp("2025-01-01 15:00:00"),
            pd.Timestamp("2025-01-01 17:00:00"),
            pd.Timestamp("2025-01-01 20:00:00"),
        ],
        "race_date": [
            pd.Timestamp("2025-01-01 00:00:00"),
            pd.Timestamp("2025-01-01 00:00:00"),
            pd.Timestamp("2025-01-01 00:00:00"),
        ],
        "horse_id": [1, 2, 3],
        "horse_name": ["Horse A", "Horse B", "Horse C"],
        "selection_type": ["BACK", "BACK", "BACK"],
        "market_type": ["WIN", "WIN", "PLACE"],
        "market_id": ["1", "2", "3"],
        "selection_id": [1, 2, 3],
        "requested_odds": [3.0, 5.0, 7.0],
        "valid": [True, True, True],
        "invalidated_at": [pd.NaT, pd.NaT, pd.NaT],
        "invalidated_reason": ["", "", ""],
        "size_matched": [0.0, 0.0, 0.0],
        "average_price_matched": [np.nan, np.nan, np.nan],
        "cashed_out": [False, False, False],
        "fully_matched": [False, False, True],
        "customer_strategy_ref": ["selection", "selection", "selection"],
        "processed_at": [
            pd.Timestamp("2025-01-01 12:00:00"),
            pd.Timestamp("2025-01-01 17:00:00"),
            pd.Timestamp("2025-01-01 18:00:00"),
        ],
        "minutes_to_race": [-10, 10, 20],
        "back_price_1": [np.nan, 5.0, 7.0],
        "back_price_1_depth": [np.nan, 100.0, 100.0],
        "back_price_2": [np.nan, 4.8, 6.8],
        "back_price_2_depth": [np.nan, 100.0, 100.0],
        "lay_price_1": [np.nan, 5.2, 7.2],
        "lay_price_1_depth": [np.nan, 100.0, 100.0],
        "lay_price_2": [np.nan, 5.4, 7.4],
        "lay_price_2_depth": [np.nan, 100.0, 100.0],
        "eight_to_seven_runners": [False, False, True],
        "short_price_removed_runners": [False, False, False],
        "average_price_matched_selections": [np.nan, np.nan, np.nan],
        "size_matched_selections": [0.0, 0.0, 0.0],
        "customer_strategy_ref_selections": ["selection", "selection", "selection"],
        "average_price_matched_betfair": [np.nan, np.nan, np.nan],
        "size_matched_betfair": [0.0, 0.0, 0.0],
        "customer_strategy_ref_betfair": [np.nan, np.nan, np.nan],
    }

    return pd.DataFrame(
        {
            **requests_dict,
            **(requests_overrides or {}),
        }
    )

In [ ]:
import pandas as pd
from api_helpers.clients import S3Client, BetFairClient
from api_helpers.clients.betfair_client import BetFairClient, BetFairOrder


@dataclass
class TradeRequest:
    valid_bets: bool
    info: str
    selections_data: pd.DataFrame | None = None
    orders: list[BetFairOrder] | None = None
    cash_out_market_ids: list[str] | None = None


def calculate_trade_positions(
    stake_size: int,
    requests_data: pd.DataFrame,
    now_timestamp: pd.Timestamp,
) -> TradeRequest:
    upcoming_bets = check_bets_in_next_hour(requests_data)
    if upcoming_bets.empty:
        tr = TradeRequest(
            valid_bets=False,
            info="No bets in the next hour",
        )
        I(tr.info)
        return tr

    valid_bets = filter_valid_bets(upcoming_bets)
    if valid_bets.empty:
        tr = TradeRequest(
            valid_bets=False,
            info="No valid bets to process",
        )
        I(tr.info)
        return tr

    matched_cash_out_market_ids = get_invalidated_fully_matched_bets_market_ids(
        valid_bets
    )
    if matched_cash_out_market_ids:
        bet_data = extract_invalidated_fully_matched_bets(
            valid_bets, matched_cash_out_market_ids
        )
    bet_data = mark_invalid_bets(valid_bets, now_timestamp)

    bet_data = mark_fully_matched_bets(bet_data, stake_size, now_timestamp)
    bet_data = set_new_size_and_price(bet_data, stake_size)
    bet_data = check_odds_available(bet_data)

    if bet_data.empty:
        tr = TradeRequest(
            valid_bets=False,
            info="Odds/Size not available for any bets",
        )
        I(tr.info)
        return tr

    orders, cash_out_market_ids = create_bet_data(bet_data)
    bet_data = update_selections_data(bet_data)

    return TradeRequest(
        valid_bets=True,
        info="Bets requested!",
        selections_data=bet_data,
        orders=orders,
        cash_out_market_ids=list(
            set(cash_out_market_ids + matched_cash_out_market_ids)
        ),
    )


def check_bets_in_next_hour(data: pd.DataFrame) -> pd.DataFrame:
    return data[(data["minutes_to_race"] < 60) & (data["minutes_to_race"] > 0)]


def update_selections_data(data: pd.DataFrame) -> pd.DataFrame:
    data = data.assign(
        average_price_matched=data["average_price_matched"].round(2),
        size_matched=data["size_matched_betfair"].round(2),
        customer_strategy_ref=data["customer_strategy_ref_betfair"].round(2),
    )
    return data.filter(items=SELECTION_COLS)


def filter_valid_bets(data: pd.DataFrame) -> pd.DataFrame:
    return data[(data["valid"] == True) & (data["cashed_out"] == False)]


def mark_invalid_bets(data: pd.DataFrame, now_timestamp: pd.Timestamp) -> pd.DataFrame:
    conditions = [
        (data["eight_to_seven_runners"] == True) & (data["market_type"] == "PLACE"),
        (data["short_price_removed_runners"] == True),
    ]

    return data.assign(
        valid=np.select(
            conditions,
            [False, False],
            default=True,
        ),
        invalidated_reason=np.select(
            conditions,
            ["Invalid 8 to 7 Place", "Invalid Short Price Removed"],
            default="Valid Bet",
        ),
        invalidated_at=np.select(
            conditions,
            [now_timestamp] * 2,
            default=pd.NaT,
        ),
        processed_at=now_timestamp,
        cash_out=np.select(
            conditions,
            [True, True],
            default=False,
        ),
    )


def extract_invalidated_fully_matched_bets(
    data: pd.DataFrame, market_ids: list[str]
) -> pd.DataFrame:
    return data[(data["market_id"].isin(market_ids))]


def get_invalidated_fully_matched_bets_market_ids(data: pd.DataFrame) -> list[str]:
    return (
        data[(data["valid"] == False) & (data["fully_matched"] == True)]["market_id"]
        .unique()
        .tolist()
    )


def mark_fully_matched_bets(
    data: pd.DataFrame, stake_size: float, now_timestamp: pd.Timestamp
) -> pd.DataFrame:
    data = data.assign(
        staked_minus_target=np.select(
            [
                (data["selection_type"] == "BACK"),
                (data["selection_type"] == "LAY"),
            ],
            [
                (stake_size - data["size_matched_betfair"]),
                (
                    (stake_size * 1.5)
                    - (
                        data["size_matched_betfair"]
                        * (data["average_price_matched_betfair"] - 1)
                    )
                ),
            ],
            default=0,
        )
    )
    data = data.assign(
        fully_matched=np.where(
            data["fully_matched"] == True,  # If already True, keep it True
            True,
            np.where(
                data["staked_minus_target"] > 1, False, True
            ),  # Otherwise, calculate normally
        ),
        processed_at=now_timestamp,
    )

    return data


def set_new_size_and_price(data: pd.DataFrame, stake_size: float) -> pd.DataFrame:
    conditions = [
        (data["selection_type"] == "BACK") & (data["size_matched"] > 0),
        (data["selection_type"] == "LAY") & (data["size_matched"] > 0),
        (data["selection_type"] == "BACK") & (data["size_matched"] == 0),
        (data["selection_type"] == "LAY") & (data["size_matched"] == 0),
    ]

    data = data.assign(
        remaining_size=np.select(
            conditions,
            [
                stake_size - data["size_matched"],
                (
                    (stake_size * 1.5)
                    - (data["average_price_matched"] - 1) * data["size_matched"]
                )
                / (data["lay_price_1"] - 1),
                stake_size,
                (stake_size * 1.5) / (data["lay_price_1"] - 1),
            ],
        ),
        amended_average_price=np.select(
            conditions,
            [
                (
                    (
                        (data["average_price_matched"] * data["size_matched"])
                        + (data["back_price_1"] * (stake_size - data["size_matched"]))
                    )
                    / stake_size
                ),
                (
                    (stake_size * 1.5)
                    / (
                        (
                            (
                                (stake_size * 1.5)
                                - (data["average_price_matched"] - 1)
                                * data["size_matched"]
                            )
                            / (data["lay_price_1"] - 1)
                        )
                        + data["size_matched"]
                    )
                    + 1
                ).round(2),
                data["back_price_1"],
                data["lay_price_1"],
            ],
        ),
    )
    data = data.assign(
        remaining_size=data["remaining_size"].round(2),
        amended_average_price=data["amended_average_price"].round(2),
    )

    return data


def check_odds_available(data: pd.DataFrame) -> pd.DataFrame:
    data = data.assign(
        available_odds=np.select(
            [
                (data["selection_type"] == "BACK")
                & (data["amended_average_price"] >= data["requested_odds"])
                & (data["back_price_1_depth"] >= data["remaining_size"]),
                (data["selection_type"] == "LAY")
                & (data["amended_average_price"] <= data["requested_odds"])
                & (data["lay_price_1_depth"] >= data["remaining_size"]),
            ],
            [True, True],
            default=False,
        )
    )
    return data[data["available_odds"] == True]


def create_bet_data(data: pd.DataFrame) -> pd.DataFrame:
    cash_out_market_ids = data[data["cash_out"] == True]["market_id"].unique().tolist()
    bets = data[data["cash_out"] == False]

    orders = []

    for i in bets.itertuples():
        if i.selection_type == "BACK":
            order = BetFairOrder(
                size=i.remaining_size,
                price=i.back_price_1,
                market_id=i.market_id,
                selection_id=i.selection_id,
                side=i.selection_type,
                strategy="mvp",
            )
            orders.append(order)
        elif i.selection_type == "LAY":
            order = BetFairOrder(
                size=i.remaining_size,
                price=i.lay_price_1,
                market_id=i.market_id,
                selection_id=i.selection_id,
                side=i.selection_type,
                strategy="mvp",
            )
            orders.append(order)
        else:
            raise ValueError(f"Invalid selection type: {i.selection_type}")

    return orders, cash_out_market_ids


def trade_markets(
    requests_data: pd.DataFrame,
    s3_client: S3Client,
    betfair_client: BetFairClient,
    now_timestamp: pd.Timestamp,
):
    trades: TradeRequest = calculate_trade_positions(
        stake_size=10,
        requests_data=requests_data,
        now_timestamp=now_timestamp,
    )
    if trades.cash_out_market_ids:
        betfair_client.cash_out_bets(trades.cash_out_market_ids)
    for order in trades.orders:
        result: OrderResult = betfair_client.place_order(order)
        if result.success:
            I(f"Order placed successfully: {order}")
        else:
            W(f"Failed to place order: {order}, Error: {result.message}")

    s3_client.store_data(trades.selections_data[SELECTION_COLS], paths.selections)

In [96]:
requests_data = create_test_data()
requests_data

,id,timestamp,race_id,race_time,race_date,horse_id,horse_name,selection_type,market_type,market_id,...,lay_price_2,lay_price_2_depth,eight_to_seven_runners,short_price_removed_runners,average_price_matched_selections,size_matched_selections,customer_strategy_ref_selections,average_price_matched_betfair,size_matched_betfair,customer_strategy_ref_betfair
0,1,2025-01-01 12:00:00,1,2025-01-01 15:00:00,2025-01-01,1,Horse A,BACK,WIN,1,...,NaN,NaN,False,False,NaN,0.0,selection,NaN,0.0,NaN
1,2,2025-01-01 17:00:00,2,2025-01-01 17:00:00,2025-01-01,2,Horse B,BACK,WIN,2,...,5.4,100.0,False,False,NaN,0.0,selection,NaN,0.0,NaN
2,3,2025-01-01 18:00:00,3,2025-01-01 20:00:00,2025-01-01,3,Horse C,BACK,PLACE,3,...,7.4,100.0,True,False,NaN,0.0,selection,NaN,0.0,NaN


In [97]:
now_timestamp = pd.Timestamp("2025-01-01 18:00:00")
stake_size = 10

s3_client = TestS3Client()
bf_client = TestBetfairClient()
trade_markets(
    requests_data=requests_data,
    s3_client=s3_client,
    betfair_client=bf_client,
    now_timestamp=now_timestamp,
)

2025-05-30T09:08:41Z | INFO - Order placed successfully: BetFairOrder(size=10.0, price=5.0, selection_id=2, market_id='2', side='BACK', strategy='mvp')


[BetFairOrder(size=10.0, price=5.0, selection_id=2, market_id='2', side='BACK', strategy='mvp')]


In [94]:
s3_client.stored_data[0]["object_path"]

'today/2025_05_30/trader_data/selections.parquet'

In [95]:
s3_client.stored_data[0]["data"]

,id,timestamp,race_id,race_time,race_date,horse_id,horse_name,selection_type,market_type,market_id,...,requested_odds,valid,invalidated_at,invalidated_reason,size_matched,average_price_matched,cashed_out,fully_matched,customer_strategy_ref,processed_at
1,2000,2025-01-01 17:00:00,2,2025-01-01 17:00:00,2025-01-01,2,Horse B,BACK,WIN,2,...,5.0,True,NaT,Valid Bet,0.0,NaN,False,False,NaN,2025-01-01 18:00:00
2,3000,2025-01-01 18:00:00,3,2025-01-01 20:00:00,2025-01-01,3,Horse C,BACK,PLACE,3,...,7.0,False,2025-01-01 18:00:00,Invalid 8 to 7 Place,0.0,NaN,False,True,NaN,2025-01-01 18:00:00


In [26]:
print_dataframe_for_testing(bet_data.selections_data)

pd.DataFrame({
'id':['2000', '3000'],
'timestamp':[pd.Timestamp('2025-01-01 17:00:00'), pd.Timestamp('2025-01-01 18:00:00'),],
'race_id':[2, 3],
'race_time':[pd.Timestamp('2025-01-01 17:00:00'), pd.Timestamp('2025-01-01 20:00:00'),],
'race_date':[pd.Timestamp('2025-01-01 00:00:00'), pd.Timestamp('2025-01-01 00:00:00'),],
'horse_id':[2, 3],
'horse_name':['Horse B', 'Horse C'],
'selection_type':['BACK', 'BACK'],
'market_type':['WIN', 'PLACE'],
'market_id':['2', '3'],
'selection_id':[2, 3],
'requested_odds':[5.0, 7.0],
'valid':[True, False],
'invalidated_at':[NaT, Timestamp('2025-01-01 18:00:00')],
'invalidated_reason':['Valid Bet', 'Invalid 8 to 7 Place'],
'size_matched':[0.0, 0.0],
'average_price_matched':[nan, nan],
'cashed_out':[False, False],
'fully_matched':[False, True],
'customer_strategy_ref':[nan, nan],
'processed_at':[pd.Timestamp('2025-01-01 18:00:00'), pd.Timestamp('2025-01-01 18:00:00'),],
'cash_out':[False, True],
})


In [27]:
bet_data.cash_out_market_ids

['3']

In [131]:
bet_data.orders

[BetFairOrder(size=10.0, price=5.0, selection_id=2, market_id='2', side='BACK', strategy='mvp')]

In [157]:
list(set([1] + [1]))

[1]